# Angular Distribution of Cascade/Muon Tracks Fit

Muon tracks, as well as electro magnetic and hadronic cascades have been
parameterized in Leif's thesis [1]_ using the following equation:

$$
\frac{1}{N}\frac{dN}{d\Omega} = a\exp(b|\cos(\theta_c) - x|^c) + d
$$

Unfortunately, this function is not suitable for inverse CDF sampling, as the
inverse is too complex. IceCube uses a similar looking function, that can be 
used for inverse CDF sampling:

$$
p(x) \sim \exp(-bx^a)x^{a-1}
$$

The aim of this notebook is to remap Leif's fittings to this new function.

[1] L. Raedel "Simulation Studies of the Cherenkov Light Yield from Relativistic
    Particles in High-Energy Neutrino Telescopes with Geant 4" (2012)

## Muons and EM Cascades

The original parameters depend on the initial energy of the primary particle:

$$
p_i = \lambda_0 + \lambda_1\cdot\ln\left(\frac{E}{\text{GeV}}\right)
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from dataclasses import dataclass, astuple

In [ ]:
@dataclass
class EnergyFit:
    a0: float
    a1: float
    b0: float
    b1: float
    c0: float
    c1: float
    d0: float
    d1: float

# params of muon
muon = EnergyFit(
    0.34485, 0.03145,
    -3.04160, -0.07193,
    0.69937, -0.01421,
    0.0, 0.0,
)

# params of EM cascades
electron = EnergyFit(
    4.27033, 0.0,
    -6.02527, 0.0,
    0.29887, 0.0,
    -0.00103, 0.0,
)
positron = EnergyFit(
    4.27725, 0.0,
    -6.02430, 0.0,
    0.29856, 0.0,
    -0.00104, 0.0,
)
photon = EnergyFit(
    4.25716, 0.0,
    -6.02421, 0.0,
    0.29926, 0.0,
    -0.00101, 0.0,
)

In [ ]:
@dataclass
class Params:
    a: float
    b: float
    c: float
    d: float

def evalParams(fit: EnergyFit, lnE: float) -> Params:
    return Params(
        fit.a0 + fit.a1 * lnE,
        fit.b0 + fit.b1 * lnE,
        fit.c0 + fit.c1 * lnE,
        fit.d0 + fit.d1 * lnE,
    )

In the following we wont be using the a parameter, as it can be substituted by
normalization.

In [ ]:
from scipy.special import gamma, gammainc
from scipy.integrate import quad

def norm_leif(a, b, c, d):
    f = lambda x: np.exp(b*x**c) + d
    return quad(f, 0, 2)[0]

def f_leif(x, a, b, c, d):
    # normalize over x in [0, 2]
    f = -b * 2**c
    norm = -2 * (d - f**(-1/c) * gammainc(1/c, f) * gamma(1/c) / c)
    return (np.exp(b*x**c) + d) / norm

def f_new(x, a, b):
    # normalize over x in [0, 2]
    norm = (1.0 - np.exp(-b * 2**a)) / (a * b)
    return np.exp(-b*x**a)*x**(a-1.0) / norm

We try to obtain the new parameters by minimizing a error function

In [ ]:
x = np.linspace(0.1, 2.0, 200)

def error(params, fit, lnE):
    l_params = astuple(evalParams(fit, lnE))
    return np.sqrt(np.square(f_new(x, *params) - f_leif(x, *l_params)).mean())

In [ ]:
from scipy.optimize import minimize

lnE = np.linspace(0.0, np.log10(1e4))
def fit(param):
    params = [minimize(lambda y: error(y, param, E), (0.39, 2.61)) for E in lnE]
    assert all(p.success for p in params)
    return np.array([p.x for p in params])

In [ ]:
muon_fit = fit(muon)
electron_fit = fit(electron)
positron_fit = fit(positron)
photon_fit = fit(photon)

In [ ]:
plt.scatter(lnE, muon_fit[:,0], s=5, c="C0", label="muon")
plt.scatter(lnE, muon_fit[:,1], s=1, c="C0")
plt.scatter(lnE, electron_fit[:,0], s=5, c="C1", label="e-")
plt.scatter(lnE, electron_fit[:,1], s=1, c="C1")
plt.scatter(lnE, positron_fit[:,0], s=5, c="C2", label="e+")
plt.scatter(lnE, positron_fit[:,1], s=1, c="C2")
plt.scatter(lnE, photon_fit[:,0], s=5, c="C3", label="$\gamma$")
plt.scatter(lnE, photon_fit[:,1], s=1, c="C3")
plt.legend()

Let's make some plots to verify the quality

In [ ]:
plt.plot(x, f_new(x, *electron_fit[-1,:]))
plt.plot(x, f_leif(x, *astuple(evalParams(electron, lnE[-1]))), "C0--", label="e-")
plt.plot(x, f_new(x, *muon_fit[-1,:]))
plt.plot(x, f_leif(x, *astuple(evalParams(muon, lnE[-1]))), "C1--", label="mu")
plt.legend()

Looks good to me!

All electro magnetic cascades have almost the same constant shape parameters as
expected:

In [ ]:
print(f"electron: {electron_fit.mean(0)} +/- {electron_fit.var(0)}")
print(f"positron: {positron_fit.mean(0)} +/- {positron_fit.var(0)}")
print(f"photon:   {photon_fit.mean(0)} +/- {photon_fit.var(0)}")

For the muon we will make a linear fit

In [ ]:
from scipy.stats import linregress

muon_a_fit = linregress(lnE, muon_fit[:,0])
muon_b_fit = linregress(lnE, muon_fit[:,1])

print(muon_a_fit)
print(muon_b_fit)

In [ ]:
plt.scatter(lnE, muon_fit[:,0], s=5)
plt.plot(lnE, muon_a_fit.slope * lnE + muon_a_fit.intercept, "C0")
plt.scatter(lnE, muon_fit[:,1], s=5)
plt.plot(lnE, muon_b_fit.slope * lnE + muon_b_fit.intercept, "C1")

## Hadronic Cascades

For hadronic showers, the energy dependency is slightly different fitted:

$$
p_i = \lambda_i\left(\ln\frac{E_0}{\text{GeV}}\right)^{\kappa_i}
$$

In [ ]:
pi_plus = EnergyFit(
    0.25877, -1.05372,
    -3.34355, -0.22303,
    0.70633, 0.34407,
    0.08572, 1.90632,
)
pi_minus = EnergyFit(
    0.25915, -1.05539,
    -3.29885, -0.22989,
    0.71082, 0.34857,
    0.11207, 2.05247,
)
K0_long = EnergyFit(
    0.25015, -1.06819,
    -3.33393, -0.22403,
    0.76039, 0.38042,
    0.14898, 2.19057,
)
proton = EnergyFit(
    0.13966, -1.30159,
    -2.82378, -0.29381,
    0.91092, 0.45380,
    0.13845, 2.02526,
)
anti_proton = EnergyFit(
    0.08111, -1.52203,
    -2.47748, -0.34737,
    1.16940, 0.56291,
    0.18410, 2.07564,
)
neutron = EnergyFit(
    0.11829, -1.37902,
    -2.75135, -0.30581,
    0.99563, 0.49587,
    0.18446, 2.16233,
)

Unfortunately, as it turns out the fit parameters in [1] are wrong. Luckily, the
parameters for specific energies (from which the fits allegedly have been
derived) are also provided, so we can use them instead.

In [ ]:
lnE = np.log10(np.array([100.0, 300.0, 700.0, 1000.0, 3000.0, 7000.0, 10000.0]))

#       a         b         c         d
pi_plus = np.array([
    [1.25808, -4.67573, 0.420470, 0.00451],
    [1.58251, -4.92785, 0.390083, 0.00335],
    [1.91994, -5.10950, 0.364429, 0.00248],
    [2.02966, -5.17145, 0.358579, 0.00225],
    [2.34377, -5.32730, 0.344146, 0.00158],
    [2.55229, -5.42131, 0.336258, 0.00121],
    [2.65911, -5.46872, 0.332784, 0.001049],
])
pi_minus = np.array([
    [1.22269, -4.63988, 0.42351, 0.00479],
    [1.62451, -4.93556, 0.38430, 0.00329],
    [1.94942, -5.12435, 0.36260, 0.00241],
    [2.02955, -5.17346, 0.35890, 0.00221],
    [2.35183, -5.32977, 0.34368, 0.00157],
    [2.59001, -5.43551, 0.33479, 0.00113],
    [2.64348, -5.46018, 0.33312, 0.00106],
])
K0_long = np.array([
    [1.17069, -4.64587, 0.43575, 0.00525],
    [1.64024, -4.94911, 0.38360, 0.00321],
    [1.91878, -5.11221, 0.36473, 0.00252],
    [2.03573, -5.17383, 0.35825, 0.00221],
    [2.31676, -5.31189, 0.34503, 0.00162],
    [2.51987, -5.40596, 0.33735, 0.00125],
    [2.66630, -5.46964, 0.33240, 0.00101],
])
proton = np.array([
    [0.94935, -4.36386, 0.46432, 0.00617],
    [1.35371, -4.72620, 0.40622, 0.00424],
    [1.66206, -4.96435, 0.38204, 0.00316],
    [1.77639, -5.02380, 0.37288, 0.00285],
    [2.07634, -5.18953, 0.35559, 0.00213],
    [2.37733, -5.34039, 0.34257, 0.00151],
    [2.49654, -5.39624, 0.33821, 0.00130],
])
anti_proton = np.array([
    [0.77685, -4.15854, 0.50388, 0.00757],
    [1.13807, -4.55514, 0.43347, 0.00518],
    [1.44369, -4.79691, 0.39771, 0.00393],
    [1.55943, -4.87633, 0.38762, 0.00349],
    [1.96817, -5.12279, 0.36035, 0.00236],
    [2.25264, -5.27689, 0.34730, 0.00175],
    [2.32799, -5.31316, 0.34414, 0.00161],
])
neutron = np.array([
    [0.90987, -4.34559, 0.47466, 0.00667],
    [1.28366, -4.68971, 0.41638, 0.00446],
    [1.61148, -4.92327, 0.38495, 0.00331],
    [1.75390, -5.00762, 0.37414, 0.00289],
    [2.12352, -5.21533, 0.35345, 0.00201],
    [2.39421, -5.34766, 0.34186, 0.00148],
    [2.47264, -5.38598, 0.33911, 0.00134],
])

In [ ]:
def error(new_p, old_p):
    return np.sqrt(np.square(f_new(x, *new_p) - f_leif(x, *old_p)).mean())

def fit(params):
    fits = [minimize(lambda y: error(y, p), (0.39, 2.61)) for p in params]
    assert all(p.success for p in fits)
    return np.array([p.x for p in fits])

In [ ]:
pi_plus_fit = fit(pi_plus)
pi_minus_fit = fit(pi_minus)
K0_long_fit = fit(K0_long)
proton_fit = fit(proton)
anti_proton_fit = fit(anti_proton)
neutron_fit = fit(neutron)

In [ ]:
plt.scatter(lnE, pi_plus_fit[:,0], s=5, c="C0", label="pi+")
plt.scatter(lnE, pi_plus_fit[:,1], s=5, c="C0")
plt.scatter(lnE, pi_minus_fit[:,0], s=5, c="C1", label="pi-")
plt.scatter(lnE, pi_minus_fit[:,1], s=5, c="C1")
plt.scatter(lnE, K0_long_fit[:,0], s=5, c="C2", label="K0")
plt.scatter(lnE, K0_long_fit[:,1], s=5, c="C2")
plt.scatter(lnE, proton_fit[:,0], s=5, c="C3", label="p")
plt.scatter(lnE, proton_fit[:,1], s=5, c="C3")
plt.scatter(lnE, anti_proton_fit[:,0], s=5, c="C4", label="p-")
plt.scatter(lnE, anti_proton_fit[:,1], s=5, c="C4")
plt.scatter(lnE, neutron_fit[:,0], s=5, c="C5", label="n")
plt.scatter(lnE, neutron_fit[:,1], s=5, c="C5")
plt.legend()

Again, make some plots to verify

In [ ]:
plt.plot(x, f_new(x, *pi_plus_fit[-1,:]))
plt.plot(x, f_leif(x, *pi_plus[-1,:]), "C0--", label="pi+")
plt.plot(x, f_new(x, *neutron_fit[0,:]))
plt.plot(x, f_leif(x, *neutron[0,:]), "C1--", label="n")
plt.plot(x, f_new(x, *K0_long_fit[1,:]))
plt.plot(x, f_leif(x, *K0_long[1,:]), "C2--", label="K0")
# plt.yscale("log")
plt.legend()

Not perfect, but it'll do for now.

Finally, let's fit the energy dependency on the fit params.
We try both linear and a power law. The latter is used in [1]

In [ ]:
def lin_fit(p):
    a_fit = linregress(lnE, p[:, 0])
    b_fit = linregress(lnE, p[:, 1])

    print(a_fit)
    print(b_fit)

lin_fit(pi_plus_fit)
lin_fit(pi_minus_fit)
lin_fit(K0_long_fit)
lin_fit(proton_fit)
lin_fit(anti_proton_fit)
lin_fit(neutron_fit)

In [ ]:
from scipy.optimize import curve_fit

def f_power(x, a, b):
    return a * np.exp(x)**b

def power_fit(p):
    popt, pcov = curve_fit(f_power, lnE, p[:, 0], (1.0, 1.0))
    print(f"{popt}; {np.sqrt(np.diag(pcov))}")
    popt, pcov = curve_fit(f_power, lnE, p[:, 1], (1.0, 1.0))
    print(f"{popt}; {np.sqrt(np.diag(pcov))}")

power_fit(pi_plus_fit)
power_fit(pi_minus_fit)
power_fit(K0_long_fit)
power_fit(proton_fit)
power_fit(anti_proton_fit)
power_fit(neutron_fit)

A power law fit does not look that much better, so we use the linear one to keep
the code more simple.